<a href="https://colab.research.google.com/github/Cehiim/TeoriaDosGrafos/blob/main/ProjetoFinal/ProjetoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Informações gerais

## Tema
* Aplicação com busca semântica para representação de grafos.

## Integrantes
* Cesar Hideki Imai - 10402758.
* João Victor Dallapé Madeira - 10400725.
* David Varão Lima Bentes Pessoa - 10402647.
* André Franco Ranieri - 10390470.

# Setup

## Integração dos pacotes

O pacote `vectordb2` é usado para armazenar e recuperar textos usando técnicas de *chunking* (segmentação de texto), *embedding* (conversão de texto para vetores numéricos) e busca vetorial.

[Referência](https://vectordb.com/)

In [ ]:
%pip install vectordb2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

O pacote `requests` é ser usado para recuperar o arquivo por meio de requisição em HTTP.

In [ ]:
%pip install requests

O pacote `network pyvis` é usado para criar uma visualização interativa de grafos.

[Referência](https://pyvis.readthedocs.io/en/latest/documentation.html)

In [ ]:
%pip install networkx pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.7 MB/s eta 0:00:00


## Importação das bibliotecas

In [ ]:
from vectordb import Memory
import requests
from pyvis.network import Network
import os # Será usado métodos para limpar o terminal para atualizar a interface em cada iteração do sistema
import time # Será usado método de espera para atualizar a interface gradualmente
from ipywidgets import widgets # Será usado para criar interfaces de usuário interativas com widgets
from IPython.display import display, clear_output # Será usado para mostrar interações visuais de maneira mais sofisticada

## Classes

### GrafoNDR

In [106]:
# Grafo como uma matriz de adjacência não-direcionado rotulado
class GrafoNDR(): # Ex 8
    TAM_MAX_DEFAULT = 100 # qtde de vértices máxima default
    # construtor da classe grafo
    def __init__(self, n=TAM_MAX_DEFAULT):
        self.n = n # número de vértices
        self.m = 0 # número de arestas
        # matriz de adjacência
        self.adj = [[0 for i in range(n)] for j in range(n)]

    def insereA(self, v, w, p):
        if(v == w or self.adj[v][w] != 0):
            return

        else:
            self.adj[v][w] = p
            self.adj[w][v] = p
            self.m += 1  # atualiza qtd arestas

# remove uma aresta v->w do Grafo
    def removeA(self, v, w):
        if(v == w or self.adj[v][w] == 0):
            return
        # testa se temos a aresta
        else:
            self.adj[v][w] = 0
            self.adj[w][v] = 0
            self.m -= 1  # atualiza qtd arestas

    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f"Adj[{i:2d},{w:2d}] = {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )


	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    # Apresentando apenas os valores 0 ou 1
    def showMin(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f" {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

    def insereV(self):
        for i in range(self.n):
            self.adj[i].append(0)
        self.n += 1
        self.adj.append([0]*self.n)

    def removeV(self, vertice):
        if(vertice >= self.n or vertice < 0):
            return False

        for i in range(self.n - 1):
            if(i >= vertice and i != self.n-1): # Substitui as conexões do vértice a ser retirado e
                self.adj[i] = self.adj[i+1]     # os vértices posteriores a ele com as conexões do próximo vértice

            self.removeA(i,vertice)
            self.adj[i].pop(vertice) # Remove o vértice escolhido da linha da matriz
        self.adj.pop() # Remove a última linha da matriz
        self.n -= 1
        return True

    def dfs(self, visitados, vertice): # Depth First Search
        visitados[vertice] = True
        for i in range(self.n):
            if(self.adj[vertice][i] != 0 and visitados[i] == False): # Caso haja acesso para um próximo vértice que não foi visitado
                self.dfs(visitados, i)

    def conexidade(self):
        for i in range(self.n):
            visitados = [False] * self.n
            self.dfs(visitados, i)
            if(all(visitados)): # Caso todos tenham sido visitados
                return "O grafo é conexo"
        return "O grafo não é conexo"

    def EhAdjacente(self, v, x): #verifica se o vértice v é adjacente a x
        if self.adj[v][x] != 0:
            return True
        else:
            return False

    def coloreV(self):
        lista_colorida = self.n * [0]
        n_cores = 0
        for i in range(self.n):
            other_colors = []
            for j in range(self.n):
                if self.EhAdjacente(i, j) and lista_colorida[j] != 0:
                    other_colors.append(lista_colorida[j])

            if other_colors == []:
                lista_colorida[i] = 1

            elif other_colors != []:
                for k in range(1, self.n):
                    if k not in other_colors:
                        if(k > n_cores):
                            n_cores = k

                        lista_colorida[i] = k
                        break

        info = [n_cores,lista_colorida]

        return info

### Memory

Aqui é utilizado a biblioteca VectorDB para criar uma memória virtual.

```
memoria = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
```

- `chunking_strategy` define a estratégia de fragmentação dos dados. No modo "sliding_window", os dados são divididos em *chunks* (pedaços de texto) de tamanho fixo.

- `window_size` define a quantidade de palavras que um *chunk* representa. Neste caso, cada *chunk* representa uma palavra.

- `overlap` define quantos elementos de sobreposição existirão entre os *chunks* adjacentes. Neste caso, não haverá sobreposição já que as palavras usadas não formam frases, logo são independentes uma das outras.

### Network

Aqui é utilizado a biblioteca Pyvis para criar uma instância para visualizar redes/grafos de forma interativa.

```
net = Network(notebook=True, cdn_resources='remote', directed=False)
```
* `notebook=True`: Esta opção indica que a visualização da rede será exibida diretamente em um notebook Jupyter.

* `cdn_resources='remote'`: Esta opção especifica que os recursos necessários (como bibliotecas JavaScript e CSS) serão carregados de um Content Delivery Network (CDN) remoto. Isso pode ajudar a reduzir o tempo de carregamento e garantir que seja usado as versões mais recentes desses recursos.

* `directed=False`: Especifica para que o grafo não seja direcionado.

## Funções auxiliares

### Cria vértice

In [88]:
def criaVertice(palavra, indice):
  vertice = {
      "palavra": palavra, # Recupera cada palavra e tira o "\n"
      "indice": indice,
  }
  return vertice

### Lê arquivo

Os dados do documento são importados e guardados na variável `dados`.

In [89]:
def leArquivoHTTP(url):
  arquivo = requests.get(url).text

  lista = arquivo.split() # Distribui cada elemento do arquivo numa lista
  n_palavras = int(lista.pop(0)) # Separa o número de palavras (primeira linha do arquivo)
  vertices = []
  for i in range(n_palavras):
    vertice = criaVertice(lista[i], i)
    vertices.append(vertice)

  dados = [n_palavras]
  dados.append(vertices)

  return dados

In [90]:
def leArquivo(origem):
  try:
    with open(origem, 'r', encoding='utf-8') as arquivo:
      n_palavras = int(arquivo.readline()) # Recupera o número de palavras (primeira linha do arquivo)

      vertices = []
      for i in range(n_palavras):
        vertice = criaVertice(arquivo.readline().strip(), i)
        vertices.append(vertice)

    dados = [n_palavras]
    dados.append(vertices)

    return dados

  except FileNotFoundError:
      print("[Erro: Arquivo não encontrado]")

### Embedding

Cada palavra é convertida para um vetor numérico e guardada na memória.

In [91]:
def embedding(memoria, n_palavras, vertices): # Método para fazer o embedding e inserção na memória de todas as palavras
  for i in range(n_palavras):
    memoria.save(vertices[i]["palavra"])

### Busca vetorial

Quanto menor é a distância, maior é a proximidade semântica.

In [92]:
def buscaVetorial(memoria, palavra): # Método para retornar os quatro elementos com maior proximidade semântica de uma palavra
  busca = memoria.search(palavra, top_n=4)
  return busca

### Busca índice

Busca um índice através da palavra dentro da lista de vértices.

In [93]:
def buscaIndice(n_palavras, vertices, palavra):
  for i in range(n_palavras):
    if(vertices[i]["palavra"] == palavra):
      return vertices[i]["indice"]
  return -1

### Busca palavra

Busca uma palavra através do índice dentro da lista de vértices.

In [94]:
def buscaPalavra(n_palavras, vertices, indice):
  if(indice >= n_palavras or indice < 0):
    return "[Erro: índice inválido]"
  for i in range(n_palavras):
    if(vertices[i]["indice"] == indice):
      return vertices[i]["palavra"]
  return "[Erro: índice não encontrado]"

### Integra grafo

A palavra mais próxima armazenada na memória é ela mesma, portanto para encontrar as outras três palavras mais próximas foram recuperadas as palavras de índice 1 até 4.

In [95]:
def integraGrafo(memoria, n_palavras, vertices):
  grafo = GrafoNDR(n_palavras) # Cria o grafo

  for i in range(n_palavras):
    busca = buscaVetorial(memoria, vertices[i]["palavra"])

    for j in range(1,4):
      palavra = busca[j]['chunk']
      distancia = busca[j]['distance']
      indice = buscaIndice(n_palavras, vertices, palavra)
      grafo.insereA(vertices[i]["indice"], indice, distancia)

  return grafo

### Grava dados

In [96]:
def gravaDados(n_palavras, vertices):
  with open("grafo.txt", "w") as arquivo:
    for i in range(n_palavras):
      palavra = vertices[i]["palavra"]
      arquivo.write(palavra+"\n")

### Insere vértice

In [97]:
def insereVertice(grafo, n_palavras, vertices, palavra):
  if(buscaIndice(n_palavras, vertices, palavra) == -1):
    grafo.insereV()
    vertices.append(criaVertice(palavra, n_palavras))
  else:
    print("[Erro: Palavra já existe]")

### Insere aresta

In [98]:
def insereAresta(grafo, n_palavras, vertices, origem, destino, peso):
  if(origem >= n_palavras or origem < 0):
    print("[Erro: origem não existe]")

  elif(destino >= n_palavras or destino < 0):
    print("[Erro: destino não existe]")

  else:
    grafo.insereA(origem, destino, peso)

### Remove aresta

In [99]:
def removeAresta(grafo, n_palavras, vertices, origem, destino):
  if(origem >= n_palavras or origem < 0):
    print("[Erro: origem não existe]")

  elif(destino >= n_palavras or destino < 0):
    print("[Erro: destino não existe]")

  else:
      grafo.removeA(origem, destino)

### Remove vértice

In [100]:
def removeVertice(grafo, n_palavras, vertices, removido):
  if(removido >= n_palavras or removido < 0):
    print("[Erro: vértice não existe]")

  else:
    for i in range(n_palavras - 1):
      if(i >= removido):
        vertices[i]["palavra"] = vertices[i+1]["palavra"]

      if(grafo.adj[i][removido] != 0):
        origem = vertices[i]["indice"]
        destino = removido
        removeAresta(grafo, n_palavras, vertices, origem, destino)

    grafo.removeV(removido)
    vertices.pop()

### Imprime vértices

In [101]:
def imprimeVertices(network, n_palavras, vertices):
  for i in range(n_palavras):
    network.add_node( # Adiciona vértices
        i, # Índice
        label=vertices[i]["palavra"], # Descrição do vértice
        color="yellow"
    )

### Imprime arestas

In [102]:
def imprimeArestas(network, n_palavras, vertices, grafo):
  for i in range(n_palavras):
    for j in range(n_palavras):
      distancia = grafo.adj[i][j]
      if(distancia != 0):
        if(distancia > 0 and distancia < 1):
          peso = (1-distancia) # Caso seja um valor entre a escala da distância semântica

        else:
          peso = distancia # Caso seja um valor fora do limite

        network.add_edge( # Adiciona as arestas
            i, # Origem
            j, # Destino
            value=peso, # Peso
            title=f'''
            {vertices[i]["palavra"]}
            {vertices[j]["palavra"]}
            Peso: {peso:.2f}
                          ''', # Descrição
            color="gray"
        )

### Imprime grafo

In [103]:
def imprimeGrafo(n_palavras, vertices, grafo):
  net = Network(notebook=True, cdn_resources='remote', directed=False, height="1200px", width="100%", bgcolor="black", font_color="white")
  imprimeVertices(net, n_palavras, vertices)
  imprimeArestas(net, n_palavras, vertices, grafo)
  net.barnes_hut() # Dispersa melhor os vértices
  net.show("grafo.html") # Salva o grafo

### Imprime vértices coloridos

In [104]:
def imprimeVerticesColoridos(network, n_palavras, vertices, cores, grupos):
  for i in range(n_palavras):
    num = grupos[i]
    network.add_node( # Adiciona vértices
        i, # Índice
        label=vertices[i]["palavra"], # Descrição do vértice
        color=cores[num - 1]
    )

### Imprime grafo colorido

In [105]:
def imprimeGrafoColorido(n_palavras, vertices, grafo, n_grupos, grupos):
  cores = [
      "blue",
      "green",
      "yellow",
      "red",
      "purple",
      "coral",
      "turquoise",
      "magenta",
      "caramel",
      "beige",
  ]
  n_cores = 10

  if(n_grupos > n_cores):
    return False

  else:
    net = Network(notebook=True, cdn_resources='remote', directed=False, height="1200px", width="100%", bgcolor="black", font_color="white")
    imprimeVerticesColoridos(net, n_palavras, vertices, cores, grupos)
    imprimeArestas(net, n_palavras, vertices, grafo)
    net.barnes_hut() # Dispersa melhor os vértices
    net.show("grafo_colorido.html") # Salva o grafo
    return True

# Aplicação

## Código

### Front

In [107]:
def mostraMenu():
    menu_text = widgets.HTML(value="""
    <h1>Menu:</h1>
    <ol>
        <li>Ler dados do arquivo</li>
        <li>Gravar dados no arquivo grafo.txt</li>
        <li>Inserir vértice</li>
        <li>Inserir aresta</li>
        <li>Remover vértice</li>
        <li>Remover aresta</li>
        <li>Exibir grafo</li>
        <li>Exibir matriz</li>
        <li>Apresentar a conexidade do grafo</li>
        <li>Encerrar a aplicação</li>
        <li>Buscar um índice pela palavra</li>
        <li>Buscar uma palavra pelo índice</li>
        <li>Fazer busca vetorial</li>
        <li>Exibir grafo colorido</li>
    </ol>
    """)
    display(menu_text)

### Back

In [108]:
def menu():
    memoria = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
    fim = False

    while(fim == False):
        time.sleep(1)
        clear_output(wait = True) # Limpa o terminal no Jupyter Notebook
        mostraMenu()
        choice = input()
        try:
            choice = int(choice)
            if choice == 1: # Cria grafo
                dados = leArquivoHTTP('https://raw.githubusercontent.com/Cehiim/TeoriaDosGrafos/refs/heads/main/Projeto/palavras.txt')
                #dados = leArquivo("palavras.txt")
                n_palavras = dados[0]
                vertices = dados[1]
                embedding(memoria, n_palavras, vertices)
                grafo = integraGrafo(memoria, n_palavras, vertices)
                print("Grafo criado com sucesso!")

            elif choice == 2: # Grava dados no arquivo .txt
                try:
                    gravaDados(n_palavras, vertices)
                    print("Os dados foram salvos no arquivo 'grafo.txt'.")
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 3: # Insere vértice
                palavra = input("Palavra a ser inserida: ")
                try:
                    insereVertice(grafo, n_palavras, vertices, palavra)
                    n_palavras += 1
                    print("Vértice inserido com sucesso!")

                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 4: # Insere aresta
                try:
                    origem = int(input("Insira o índice de origem: "))
                    destino = int(input("Insira o índice de destino: "))
                    peso = float(input("Insira o peso: "))
                    try:
                        insereAresta(grafo, n_palavras, vertices, origem, destino, peso)
                        print("Aresta inserida com sucesso!")

                    except NameError:
                        print("[Erro: Grafo não criado]")

                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 5: # Remove vértice
                try:
                    indice = int(input("Insira o índice do vértice: "))
                    try:
                        removeVertice(grafo, n_palavras, vertices, indice)
                        n_palavras -= 1
                        print("Vértice removido com sucesso!")

                    except NameError:
                        print("[Erro: Grafo não criado]")

                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 6: # Remove aresta
                try:
                    origem = int(input("Insira o índice de origem: "))
                    destino = int(input("Insira o índice de destino: "))
                    try:
                        removeAresta(grafo, n_palavras, vertices, origem, destino)
                        print("Aresta removida com sucesso!")

                    except NameError:
                        print("[Erro: Grafo não criado]")

                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 7: # Exibe grafo
                try:
                    imprimeGrafo(n_palavras, vertices, grafo)
                    print(f"\nGrafo não-direcionado rotulado com {grafo.n} vértices e {grafo.m} arestas\n")
                    print("O grafo visual foi criado no arquivo 'grafo.html'.")
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 8: # Exibe matriz
                try:
                    grafo.showMin()
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 9: # Apresenta a conexidade do grafo e grafo reduzido
                try:
                    if(grafo.conexidade()):
                        print("O grafo é conexo.")
                    else:
                        print("O grafo não é conexo.")
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 10: # Encerra
                fim = True
                print("Encerrando programa...")

            elif choice == 11: # Busca um índice pela palavra
                try:
                    palavra = input("Palavra a ser consultada: ")
                    indice = buscaIndice(n_palavras, vertices, palavra)
                    if(indice == -1):
                        print("[Erro: palavra não encontrada]")
                    else:
                        print(f"Índice de {palavra}: {indice}")
                        print(("\nAperte ENTER para continuar "))
                        ok = input()
                except NameError:
                    print("[Erro: Grafo não criado]")

            elif choice == 12: # Busca um índice pela palavra
                try:
                    indice = int(input("Índice a ser consultado: "))
                    try:
                        palavra = buscaPalavra(n_palavras, vertices, indice)
                        if(indice == -1):
                            print("[Erro: palavra não encontrada]")
                        else:
                            print(f"Palavra do índice {indice}: {palavra}")
                            print(("\nAperte ENTER para continuar "))
                            ok = input()
                    except NameError:
                      print("[Erro: Grafo não criado]")
                except ValueError:
                    print("[Erro: a entrada não é do tipo int]")

            elif choice == 13: # Fazer busca vetorial de uma palavra
                busca = input("Insira uma palavra: ")
                try:
                    resultado = memoria.search(busca, top_n=3)
                    print(f"\n\nBusca: {busca}\n")
                    for i in range(3):
                        palavra = resultado[i]['chunk']
                        distancia = resultado[i]['distance']
                        print(f"Palavra: {palavra}\nDistância: {distancia:.2f}\n")
                    print(("\nAperte ENTER para continuar "))
                    ok = input()
                except IndexError:
                    print("[Erro: Grafo não criado]")

            elif choice == 14: # Exibir grafo colorido
                try:
                    lista_colorida = grafo.coloreV()
                    n_cores = lista_colorida[0]
                    cores = lista_colorida[1]
                    if(imprimeGrafoColorido(n_palavras, vertices, grafo, n_cores, cores)):
                        print(f"\nGrafo não-direcionado rotulado com {grafo.n} vértices, {grafo.m} arestas e {n_cores} cores\n")
                        print("O grafo visual foi criado no arquivo 'grafo.html'.")
                        print(("\nAperte ENTER para continuar "))
                        ok = input()
                    else:
                        print("[Erro:Não é possível exibir um grafo com mais de 10 cores]")
                except NameError:
                    print("[Erro: Grafo não criado]")

            else:
                print("Opção inválida.")

        except ValueError:
            # print("[Erro: a entrada não é do tipo int]")
            pass

## Menu

In [87]:
menu()

HTML(value='\n    <h1>Menu:</h1>\n    <ol>\n        <li>Ler dados do arquivo</li>\n        <li>Gravar dados no…

10
Encerrando programa...
